<a href="https://colab.research.google.com/github/TheAcoff/chat-Fismat/blob/main/Chat_Fismat_Obsidian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ferramenta de Auxílio em Estudos

Para este código, iremos utilizar dois modelos diferentes. Um para a detecção de intenção do usuário, tendo como saída variáveis precisas que irão entrar em uma lista para serem input de um segundo modelo que terá os dados treinados para a Física Matemática.

Para o modelo1, temos a saída:
$$
output1 = \begin{pmatrix} intenção \\ tema \\ ordem \end{pmatrix}  = input2
$$




In [ ]:
import google.generativeai as genai
import hashlib
import os
from google.colab import drive,userdata
from pathlib import Path

#Exportando arquivos do Google Drive
drive.mount('/content/drive')
pasta_origem = Path('/content/drive/My Drive/IATeste')

#Configuração de API
APIKey = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=APIKey)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Funções principais
Nessa parte, deixo todas as funções que estão sendo utilizadas para a manipulação de dados. Começando com uma função para converter a saída de texto do Gemini por uma matriz.

In [ ]:
def string_para_lista(texto):
  # Remove as chaves e divide a string em elementos usando a vírgula como separador
  elementos = texto.strip("[]").split(",")

  elementos = [elemento for elemento in elementos]

  return elementos


Função destinada a extrair os textos dos arquivos:

In [ ]:
def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]
  # Add logic to read the PDF and return a list of pages here.
  pages = []
  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

Função destinada a importar todos os arquivos do Google Drive da sua pasta para o history do Gemini

In [ ]:
arquivos = []

# Looping pelos arquivos na pasta
for filename in os.listdir(pasta_origem):
    # Verifique se o arquivo é um .md
    if filename.endswith(".md"):
        # Crie o caminho completo para o arquivo
        filepath = os.path.join(pasta_origem, filename)

        # Adicione um dicionário à lista com as informações do arquivo
        arquivos.append({
            "role": "user",
            "parts": extract_pdf_pages(filepath)
        })
    if filename.endswith(".pdf"):
        # Crie o caminho completo para o arquivo
        filepath = os.path.join(pasta_origem, filename)

        # Adicione um dicionário à lista com as informações do arquivo
        arquivos.append({
            "role": "user",
            "parts": extract_pdf_pages(filepath)
        })

print(arquivos)


[{'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Algebra Linear.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Limites.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Planos.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Integral.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Rotacional.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Divergente.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Funções Especiais.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Transformada de Laplace.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Drive/IATeste/Coordenadas Cilíndricas.md ---']}, {'role': 'user', 'parts': ['--- START OF PDF $/content/drive/My Driv

### Geração de Texto
Separei dois prompts diferentes, um para criar texto novo, outro para editar um arquivo já existente.

In [ ]:
def geracao_texto_novo(detalhe, tema, ordem):
  #Modelo vai gerar uma explicação para o usuário
  output1 = chat1.send_message(f'Baseado nos textos .md e no livro, explique {ordem} {detalhe} sobre {tema}')
  print(output1.text)
  return output1.text

In [ ]:
def geracao_texto_edit(detalhe, tema, ordem,arquivo):
  #Modelo vai gerar uma explicação para o usuário
    output1 = chat1.send_message(f'Sugira modificações para o {arquivo}.md sobre {tema}. \n Faça os links dos termos mais importantes com [[]] como é feito nos arquivos enviados.  Escreva um arquivo em .md. Coloque equações em usando $. Explique {ordem} {detalhe}')
    print(output1.text)
    return output1.text

Em alguns casos, as análises podem retornar nehuma intenção do usuário em uma ordem específica, pra isso, limpamos a entrada para a sintaxe ser apenas um espaço no prompt:

In [ ]:
def variavel_vazia(variavel, entrada):
  if variavel[entrada] == 'Nenhuma':
    saida = ' '
  else:
    saida = variavel[entrada]
  return saida

Essa parte do texto faz uma análise do usuário, gerando um feedback positivo, negativo ou de saída. O usuário pode querer resumir mais o texto, detalhar ou até mesmo gerar detalhes mais específicos.

In [ ]:
def analise_usuario(prompt):
  #Modeo vai analisar a explicação do usuário e gerar novas variáveis
  output2 = chat2.send_message(prompt)
  output2M = string_para_lista(output2.text)

  feedback = output2M[0]
  ordem = variavel_vazia(output2M,1)
  detalhe = variavel_vazia(output2M,2)

  return feedback, ordem, detalhe


# Definição de Parâmetros
Nessa parte, definimos os parâmetros dos dois modelos. Seja $n$ o número do modelo, podendo ser $0$ ou $1$ para os parâmetros, então, definimos previamente: generation_config$\,n$ , safety_settings$\,n$

## Parâmetros do Modelo 0 e 2

In [ ]:
generation_config0 = {
  "temperature": 0.4,
  "top_p": 0,
  "top_k": 0,
  "max_output_tokens": 4096
}

safety_settings0 = {
    'HARASSMENT':'BLOCK_MEDIUM_AND_ABOVE',
    'HATE':'BLOCK_MEDIUM_AND_ABOVE',
    'SEXUAL':'BLOCK_MEDIUM_AND_ABOVE',
    'DANGEROUS':'BLOCK_MEDIUM_AND_ABOVE',
}

## Parâmetros do Modelo 1

In [ ]:
generation_config1 = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 12000
}

system_instruction1 = "Aja como um auxiliar para anotações no obsidian. Faça a resposta em um bloco de código."

safety_settings1 = {
    'HARASSMENT':'BLOCK_NONE',
    'HATE':'BLOCK_NONE',
    'SEXUAL':'BLOCK_NONE',
    'DANGEROUS':'BLOCK_NONE',
}

# Incializando os Modelos

In [ ]:
# Modelo para primeira avaliação do input do usuário
model0 = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config = generation_config0,
    safety_settings = safety_settings0,
    )

#Modelo principal de geração de textos
model1 = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    generation_config = generation_config1,
    safety_settings = safety_settings1,
    system_instruction = system_instruction1,
    )

#Modelo para validar a opinião do usuário com base no resultado do 'model1'
model2 = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config = generation_config0,
    safety_settings = safety_settings0,
    )

In [ ]:
chat0 = model0.start_chat(history=[
  {
    "role": "user",
    "parts": ["Aja como um identificador de intenção e ordem e me ajude a detectar o tema. \nDevemos ter dois resultados possíveis de Intenção desejadas pelo usuário: Dúvida e Edição. \nJá para ordem: Nenhuma, Resuma, Detalhe, Crie.\nPara isso, responda no seguinte formato, como se fosse declarar uma matriz em Python, sem utilizar os espaços: [Intenção,Ordem,Tema] \n---\nExemplo1: Gostaria de saber mais sobre Inteligência Artificial \n[Dúvida,Detalhe,Inteligência Artificial]\nExemplo2: Me ajude com minhas notas sobre Redes Neurais \n[Edição,Complemente,Redes Neurais] \nExemplo3: Crie uma nota sobre Produto Interno\n[Geração,Crie,Produto Interno]\nExemplo4: Me explique sobre Espaço Vetorial. \n[Dúvida,Nenhma,Espaço Vetorial] \nExemplo 5: Fale mais sobre Jupiter e suas luas \n[Dúvida,Detalhe,Luas de Júpiter] \nExemplo 6: Resume sobre crateras da Terra \n[Dúvida,Resuma,Crateras da Terra] \nExemplo7: Resuma minhas notas sobre Teclado \n[Edição,Resuma,Teclado]"]
  },
  {
    "role": "model",
    "parts": ["Ok, irei analisar as frases e fazer no modelo [Intenção,Ordem,Tema]"]
  }
])

chat1 = model1.start_chat(history = arquivos)#Fazer treinamento

chat2 = model2.start_chat(history=[
  {
    "role": "user",
    "parts": ["Aja como um identificador de aprovação do usuário. Me ajude a detectar a categoria, a ordem e o detalhe da frase do usuário. \n\nDevemos ter três tipos de categoria: aprovado, desaprovado e sair.\n\nDevemos ter três tipos de ordem: resuma, detalhe, nenhuma\n\nPara o detalhe, analise qual parte o usuário deseja uma explicação, caso categoria for desaprovado. Se for diferente de desaprovado, a resposta será nenhuma.\n\nO formato de saída deve ser como uma lista em Python: [categoria,ordem,detalhe]\n\n---\nExemplo1:  Não estou satisfeito com a resposta.\n[Desaprovado,Nenhuma,Nenhuma]\n\nExemplo2: Não gostei, resuma mais um pouco.\n[Desaprovado,Resuma,Nenhum]\n\nExemplo3: Resuma mais a parte do terceiro parágrafo do assunto\n[Desaprovou, Resuma, terceiro parágrafo do assunto]\n\nExemplo4: Obrigado, estou satisfeito\n[Aprovado,Nenhuma,Nenhuma]\n\nExemplo5: Detalhe mais sobre o capitalismo\n[Desaprovado,Detalhe,capitalismo]\n\nExemplo6: Não precisa gerar mais\n[Sair,Nenhuma,Nenhuma]\n\nExemplo7: Tudo bem, eu pergunto outra coisa\n[Sair,Nenhuma,Nenhuma]\n\nExemplo8: Explique melhor as equações\n[Desaprovado,Detalhe,equações]"]
  },
  {
    "role": "model",
    "parts": ["[Desaprovado, Nenhuma, Nenhuma]\n[Desaprovado, Resuma, Nenhuma]\n[Desaprovado, Resuma, terceiro parágrafo do assunto]\n[Aprovado, Nenhuma, Nenhuma]\n[Desaprovado, Detalhe, capitalismo]\n[Sair, Nenhuma, Nenhuma]\n[Sair, Nenhuma, Nenhuma]\n[Desaprovado, Detalhe, equações]"]
  },
  {
    "role": "user",
    "parts": ["Obrigado, não quero mais informações"]
  },
  {
    "role": "model",
    "parts": ["[Sair, Nenhuma, Nenhuma]"]
  },
])

# Código em Ação

In [ ]:
#Declaração de Variáveis
prompt = str(input('Input: '))
detalhe = str(' ')
ordem = str(' ')

while prompt != 'fim':
  #Envia o prompt e converte a saída para uma matriz.
  output0 = chat0.send_message(prompt)
  output0M = string_para_lista(output0.text)

  #Separa as variáveis da saída
  intencao = output0M[0]
  tema = output0M[2]
  ordem = variavel_vazia(output0M,1)
  # if output0M[1] == 'Nenhuma':
  #   ordem = ' '
  # else:
  #   ordem = output0M[1]

  print(output0M)

  #Verifica se a intenção é Dúvida ou Edição
  feedback = str(' ')
  if intencao == 'Dúvida':
    #Rodamos o código enquanto o usuário não estiver satisfeito:
    while feedback != ('Sair' or 'Aprovado'):
      #Geramos os textos para o usuário:
      output1 = geracao_texto_novo(detalhe, tema, ordem)
      print('finja que aqui tem uma resposta bonita da IA')

      #Coletando resposta
      prompt = str(input('Gostou do texto gerado?: '))
      feedback, ordem, detalhe = analise_usuario(prompt)

      print(output2M)

  if intencao == 'Edição':
    '''Buscar arquivo com Tema. Tem que ter var com nome do arquivo caso
    encontrado e outra pra validação'''

    if (ordem == 'Crie') or (validacao_arquivo == 'Nao encontrado'):
      while feedback != ('Sair'):
        #Geramos os textos para o usuário
        output1 = geracao_texto(detalhe,ordem,tema)
        #Coletando resposta
        prompt = str(input('Gostou do texto gerado?: '))
        feedback, ordem, detalhe = analise_usuario(prompt)
        if feedback == 'Aprovado':
          '''#Criar novo arquivo .md com as notas geradas (output1) e nome Tema'''
          feedback = ' '
          print('Criando aqruivo .md')

    else: #(ordem == resuma ou detalhe) ou (arquivo for existente)
      while feedback != ('Sair' or 'Aprovado'):
        #Geramos os textos para o usuário
        output1 = geracao_texto_edit(detalhe,ordem,tema,arquivo)
        #Coletando resposta
        prompt = str(input('Gostou do texto gerado?: '))
        feedback, ordem, detalhe = analise_usuario(prompt)
      if feedback == 'Aprovado':
        '''#Pegar o arquivo com o mesmo nome do tema e edita-lo com outpu1'''



  prompt = str(input('Input: '))









